In [8]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

In [3]:
#check related prime
def is_related_prime(k, n):
    n = int(n/2)
    for i in range(2,n):
        if k%i==0 and n%i==0:
            return False
    return True

#calculate division over F_q
def cal_div(x , q):
    if x == 0:
        return 0
    if x == 1:
        return 1
    for i in range(1, q):
        if (x*i) % q == 1:
            return i
    return 0 

In [4]:
#point over elliptic curve
class point(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        if type(y) == int:
            self.my = - self.y
    def Print(self):
        return list([self.x, self.y])
        
#Embedding plane text on the elliptic curve
class EC(object):
    # y = x^3 + a*x + b over Z_q
    def __init__(self, q, a, b):
        assert 0 < a and a < q and 0 < b and b < q and q > 2
        assert (4 * (a ** 3) + 27 * (b ** 2))  % q != 0
        self.a = a
        self.b = b
        self.q = q
        
    def Print(self):
        return [self.a, self.b, self.q]
    
    # 對於x 判斷在曲線上是否有解
    def point_on_curve(self, x):
        square_of_y = (x**3 + x*self.a + self.b) % self.q
        for i in range(self.q):
            if (i*i) % self.q == square_of_y:
                return i
        return False

class EG(object):
    def __init__(self, ec):
        self.ec = ec
        self.zero = (0, 0, 0)
    def get_point(self, m):
        p = point(0, 0)
        if self.ec.point_on_curve(m):
            py = self.ec.point_on_curve(m)
        else:
            py = "not exist"
        p = point(m, py)
        return p
    def is_valid(self, p):
        if p == self.zero: return True
        l = (p.y ** 2) % self.ec.q
        r = ((p.x ** 3) + self.ec.a * p.x + self.ec.b) % self.ec.q
        return l == r
            
    def add_on_curve(self, p1, p2):
        p1x = p1.x
        p1y = p1.y
        p2x = p2.x
        p2y = p2.y
        
        #p1 + p2 = p3
        p3 = point(0, 0)
        if p1 == point(0, 0):
            return p2
        if p2 == point(0, 0):
            return p1
        # p1 + -p1 == 0
        if p1x == p2x and (p1y != p2y or p1y == 0):
            p3 = point(0, 0)
            return p3
        # p1 + p1: use tangent line of p1 as (p1,p1) line
        if p1x != p2x:
            m = (p2y - p1y ) * cal_div((p2x - p1x), self.ec.q) % self.ec.q
            p3x = (m*m - p1x - p2x) % self.ec.q
            p3y = (m*(p1x - p3x) - p1y) % self.ec.q
            p3 = point(p3x, p3y)
            return p3
        else:
            m = (3*(p1x*p1x) + self.ec.a) * cal_div(2*p1y, self.ec.q) % self.ec.q
#             print('m:{}, a:{}, x:{}, y:{}, q:{}, un:{}, in:{}'.format(m, self.ec.a, p1x, p1y, ec.q, 2*p1.y, cal_div(2*p1y, ec.q)))
            p3x = ((m*m) - 2*p1x) % self.ec.q
#             print('m: {} , p1x: {}, p3x: {}'.format(m, p1x, p3x))
            p3y = (m*(p1x - p3x) - p1y) % self.ec.q
            p3 = point(p3x, p3y)
            return p3
    def mul_on_curve(self, k, p):
        pxx = p
        while 0 < k:
            if k & 1 == 1:
                pxx = self.add_on_curve(pxx, p)
            k, pxx = k >> 1, self.add_on_curve(pxx, pxx)
        return pxx
    
    def order_of_point(self, p):
        k = 1
        p1 = p
        p1 = self.add_on_curve(p1, p)
        while p1.y != 0:
            k = k+1
            p1 = self.add_on_curve(p1, p)
        return k+1

In [5]:
class ECDH(object):
    def __init__(self, ec, eg, kA, kB, G):
        self.ec = ec    #ellpiptic curve
        self.eg = eg
        self.kA = kA
        self.kB = kB
        self.G = G
        self.kAG = self.eg.mul_on_curve(self.kA, self.G)
        self.kBG = self.eg.mul_on_curve(self.kB, self.G)
        self.kBkAG = self.eg.mul_on_curve(self.kB, self.kAG)
 
    def encode(self, m):
        P_m = self.eg.get_point(m)
        assert type(P_m.y) == int
        kAkBG = self.eg.mul_on_curve(self.kA, self.kBG)
        return self.eg.add_on_curve(P_m, kAkBG)
    
    def decode(self, encode_m):
        inverse_kBkAG = point(self.kBkAG.x, -self.kBkAG.y)
        return self.eg.add_on_curve(encode_m, inverse_kBkAG)

In [6]:
#find random prime number of bit long 14, 20,32
def get_prime(bit_long):
    assert bit_long >= 3
    cond = True
    while cond:
        n = random.randint((2**(bit_long-1))+1, 2**bit_long)
        n_sqrt = int(np.sqrt(n))
        for i in range(2, n_sqrt+1):
            if (n%i == 0):
                break
            if (i == n_sqrt):
                if n%i != 0:
                    cond = False
    return n

In [14]:
def get_encode_message(bit_long):
    cond1 = True
    while cond1:
        p  = get_prime(bit_long)
        a  = random.randint(0, p-1)
        b  = random.randint(0, p-1)
        if EC(p,a,b):
            ec = EC(p,a,b)
            x  = random.randint(0, p-1)
            G = EG(ec).get_point(x)
            if type(G.y) == int:
                if  EG(ec).order_of_point(G)>512:
                    cond1 = False
#     print('p : {}, a : {}, b : {}, G : ({}, {}))'.format(p, a, b, G.x, G.y))
    ec = EC(p,a,b)
    eg = EG(ec)
    order = EG(ec).order_of_point(G)
    
    #find K
    K = int(p/513)
    message = [i for i in range(512)]
    x_m = []
    for m in message:
        for j in range(0, K):
            x = m*K + j
            if ec.point_on_curve(x):
                x_m.append(x)
                break
    assert len(x_m) == 512
    
    ka = random.randint(1, order-1)
    kb = random.randint(1, order-1)
    
    data = pd.DataFrame(columns = ['p', 'K', 'm', 'decode_m', 'encode_m', 'kAG.x', 'kAG.y', 'kBG.x', 'kBG.y'])
    ecdh = ECDH(ec, eg, ka, kb, G)
    
    for ii in range(512):
        if ii%100 == 0:
            print("进度:{}, {}%".format(ii, round((ii) * 100 / 512)), end="\r")
        encode_m = ecdh.encode(x_m[ii])
        decode_m = ecdh.decode(encode_m)
        assert ii == int(decode_m.x/K)
        
        data = data.append({'p': p, 'K': K, 'm': ii, 
                            'decode_m': decode_m.x , 'encode_m': encode_m.x,
                            'kAG.x': ecdh.kAG.x, 'kAG.y': ecdh.kAG.y,
                            'kBG.x': ecdh.kBG.x, 'kBG.y': ecdh.kBG.y
                           }, ignore_index=True) 
    
    return data

In [13]:
get_encode_message(14)

p : 9923, a : 2068, b : 3913, G : (9660, 2491))
19


,p,K,m,decode_m,encode_m,kAG.x,kAG.y,kBG.x,kBG.y
0,9923,19,0,1,6998,3247,6233,1395,4197
1,9923,19,1,19,2681,3247,6233,1395,4197
2,9923,19,2,38,1679,3247,6233,1395,4197
3,9923,19,3,57,9192,3247,6233,1395,4197
4,9923,19,4,76,6452,3247,6233,1395,4197
5,9923,19,5,99,5796,3247,6233,1395,4197
6,9923,19,6,114,1264,3247,6233,1395,4197
7,9923,19,7,133,8103,3247,6233,1395,4197
8,9923,19,8,152,9501,3247,6233,1395,4197
9,9923,19,9,173,5762,3247,6233,1395,4197


In [ ]:
for i in tqdm_notebook(range(500,700)):
    cond = True
    while cond :
        try:
            df   = get_encode_message(14)
            cond = False
        except:
            cond = True
    df.to_csv('data_p14/p14_{}.csv'.format(i), header=False)

HBox(children=(IntProgress(value=0, max=200), HTML(value='')))

In [18]:
count = []
cond = True
for n in range(2**13, 2**14)   :
    n_sqrt = int(np.sqrt(n))
    for i in range(2, n_sqrt+1):
        if (n%i == 0):
            break
        if (i == n_sqrt):
             if n%i != 0:
                    count.append(n)

In [19]:
len(count)

872